In [183]:
import tensorflow as tf
from keras import metrics
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras import Model
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import Input, Flatten, Dense, Dropout, GlobalAveragePooling2D

import numpy as np
import os
import shutil
import pathlib
from glob import glob

import requests 
from bs4 import BeautifulSoup
import urllib.request

import PIL
from PIL import Image

from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
%matplotlib inline

In [150]:
NUM_CLASSES = 2
BATCH_SIZE = 16
IMG_SIZE = (224,224)
LR = 0.1

## Load Lucy Images

In [122]:
# Load and convert images to .jpeg

img_dir = "/Users/johngalvin/Desktop/GitHub/Tensorflow/workspace/Lucy-Classification/images/lucy/"

for file in os.listdir(img_dir):
    if file.endswith(".JPG") or file.endswith(".jpg"):
        img = Image.open(img_dir + file)
        file_name, file_ext = os.path.splitext(file)
        new_name = file_name + ".jpeg"
        img.save(img_dir + new_name)

In [123]:
# Delete the .JPG and.jpg files

for file in os.listdir(img_dir):
    if file.endswith(".JPG") or file.endswith(".jpg"):
        path_to_file = os.path.join("/Users/johngalvin/Desktop/GitHub/Tensorflow/workspace/Lucy-Classification/images/lucy/", file)
        os.remove(path_to_file)

## Download Other Images (Not Lucy)

In [124]:
# Function to grab image urls from a url

def getdata(url): 
    r = requests.get(url) 
    return r.text 
    
htmldata = getdata("https://unsplash.com/s/photos/french-bulldog") 
soup = BeautifulSoup(htmldata, 'html.parser')
img_list = [item['src'] for item in soup.find_all("img")]

In [125]:
i = 0
while i < len(img_list):
    f_name = "/Users/johngalvin/Desktop/GitHub/Tensorflow/workspace/Lucy-Classification/images/other/" + str(i) + ".jpeg"
    urllib.request.urlretrieve(img_list[i], f_name)
    i+=1

## Train / Test Split

### Lucy Images

In [136]:
# Generate a list of the image files
lucy_image_files = glob("../images/lucy/*.jpeg")

# Strip the extensions
lucy_image_names = [name.replace(".jpeg","") for name in lucy_image_files]

# Split into train and test
lucy_train_names, lucy_test_names = train_test_split(lucy_image_names, test_size=0.2)

def batch_move_files(file_list, source_path, destination_path):
    """Moves jpeg files to destination path from source path"""
    
    for file in file_list:
        image = file + ".jpeg"
        shutil.move(image, destination_path)
    
    return

# Move the files - assumes empty train and test directories at the images directory level

source_dir = "../images/lucy/"
val_dir = "../images/val/lucy/"
train_dir = "../images/train/lucy/"
batch_move_files(lucy_test_names, source_dir, val_dir)
batch_move_files(lucy_train_names, source_dir, train_dir)

### Other Images

In [137]:
# Generate a list of the image files
other_image_files = glob("../images/other/*.jpeg")

# Strip the extensions
other_image_names = [name.replace(".jpeg","") for name in other_image_files]

# Split into train and test
other_train_names, other_test_names = train_test_split(other_image_names, test_size=0.2)

# Move the files - assumes empty train and test directories at the images directory level

source_dir = "../images/other/"
val_dir = "../images/val/other/"
train_dir = "../images/train/other/"
batch_move_files(other_test_names, source_dir, val_dir)
batch_move_files(other_train_names, source_dir, train_dir)

## Image Augmentation

In [155]:
# Create generators and preprocess input

train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input,
                                   rotation_range=20,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   zoom_range=0.2)

val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

In [156]:
# Use the generators

train_generator = train_datagen.flow_from_directory("../images/train/",
                                                    target_size=IMG_SIZE,
                                                    batch_size=BATCH_SIZE,
                                                    shuffle=True,
                                                    seed=42,
                                                    class_mode="categorical")

val_generator = test_datagen.flow_from_directory("../images/val/",
                                                    target_size=IMG_SIZE,
                                                    batch_size=BATCH_SIZE,
                                                    shuffle=False,
                                                    class_mode="categorical")

Found 221 images belonging to 2 classes.
Found 56 images belonging to 2 classes.


## Define and Fine Tune Model

In [239]:
# Instantiate base model from ResNet50 and exclude head

base_model = ResNet50(include_top=False,
                      input_shape=IMG_SIZE + (3,))

for layer in base_model.layers[:]:
    layer.trainable = False

# Create custom model

input_layer = Input(shape=IMG_SIZE + (3,))

custom_model = base_model(input_layer)
custom_model = GlobalAveragePooling2D()(custom_model)
custom_model = Dense(64, activation="relu")(custom_model)
custom_model = Dropout(0.2)(custom_model)
predictions = Dense(2, activation="softmax")(custom_model)

# Instantiate custom model

model = Model(inputs=input_layer, outputs=predictions)

# Compile model

model.compile(loss="categorical_crossentropy",
              optimizer=tf.keras.optimizers.Adam(learning_rate=LR),
              metrics=["acc"])

In [240]:
history = model.fit_generator(train_generator,
                    epochs=5,
                    validation_data=val_generator)

/var/folders/xt/wkt538cx4tbf_ssfzkgm44z40000gn/T/ipykernel_46153/4069364747.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(train_generator,


Epoch 1/5


2022-06-10 13:54:54.974824: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


14/14 [==============================] - ETA: 0s - loss: 36.0617 - acc: 0.6697

2022-06-10 13:55:09.600560: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


14/14 [==============================] - 19s 1s/step - loss: 36.0617 - acc: 0.6697 - val_loss: 1.9992 - val_acc: 0.9286
Epoch 2/5
14/14 [==============================] - 17s 1s/step - loss: 0.7261 - acc: 0.9502 - val_loss: 0.1959 - val_acc: 0.9464
Epoch 3/5
14/14 [==============================] - 16s 1s/step - loss: 0.4643 - acc: 0.9186 - val_loss: 0.1536 - val_acc: 0.9643
Epoch 4/5
14/14 [==============================] - 17s 1s/step - loss: 0.1406 - acc: 0.9729 - val_loss: 0.1230 - val_acc: 0.9643
Epoch 5/5
14/14 [==============================] - 16s 1s/step - loss: 0.1725 - acc: 0.9412 - val_loss: 0.1496 - val_acc: 0.9286


In [159]:
model.summary()

Model: "model_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_25 (InputLayer)       [(None, 224, 224, 3)]     0         
                                                                 
 resnet50 (Functional)       (None, 7, 7, 2048)        23587712  
                                                                 
 global_average_pooling2d_9   (None, 2048)             0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dense_13 (Dense)            (None, 64)                131136    
                                                                 
 dropout_12 (Dropout)        (None, 64)                0         
                                                                 
 dense_14 (Dense)            (None, 2)                 130       
                                                          

In [241]:
model.save("model.h5")

/Users/johngalvin/miniforge3/lib/python3.9/site-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)


In [242]:
model = load_model("model.h5")

In [251]:
img_path = "/Users/johngalvin/Desktop/GitHub/Tensorflow/workspace/Lucy-Classification/images/IMG_1819.jpg"

In [252]:
img = tf.keras.preprocessing.image.load_img(img_path, target_size=(224,224))

In [253]:
img_array = tf.keras.preprocessing.image.img_to_array(img)

In [254]:
expanded_img_array = np.expand_dims(img_array, axis=0)

In [255]:
preprocessed_img = preprocess_input(expanded_img_array)

In [256]:
prediction = model.predict(preprocessed_img)

In [257]:
print (prediction)

[[1.0000000e+00 1.1442813e-15]]


In [258]:
print (val_generator.class_indices)

{'lucy': 0, 'other': 1}
